# CNN & More
---
## Easy, but Important!

### Library Loading

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Data Loading

In [ ]:
(train_x, train_y), (test_x, test_y) = keras.datasets.mnist.load_data()

In [ ]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

* 데이터 살펴보기

In [ ]:
id = np.random.randint(0, 10000)

print(f'id = {id}')
print(f'다음 그림은 숫자 {test_y[id]} 입니다.')

plt.imshow(test_x[id], cmap='Greys')
plt.show()

## Data Preprocessing

* Data split
    - training set : validation set = 8 : 2
    - 재연을 위한 난수 고정 : 2023

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=2023)

* Scaling
    - min-max scaling

In [ ]:
print('max :', train_x.max(),'  min :', train_x.min())

In [ ]:
max_v, min_v = train_x.max(), train_x.min()
max_v, min_v

In [ ]:
train_x = (train_x - min_v) / (max_v - min_v)
val_x = (val_x - min_v) / (max_v - min_v)
test_x = (test_x - min_v) / (max_v - min_v)

In [ ]:
print('max :', train_x.max(),'  min :', train_x.min())

* One-Hot Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
class_n = len(np.unique(train_y))

In [ ]:
train_y = to_categorical(train_y, class_n)
val_y = to_categorical(val_y, class_n)
test_y = to_categorical(test_y, class_n)

In [ ]:
train_x.shape, train_y.shape

* 흑백 정보를 명시하기 위한 reshape

In [ ]:
train_x = np.expand_dims(train_x, axis=-1)
val_x = np.expand_dims(val_x, axis=-1)
test_x = np.expand_dims(test_x, axis=-1)

In [ ]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

## **Image Data Augmentation**

- ImageDataGenerator
- .flow( )

In [ ]:
trainIDG = ImageDataGenerator(rotation_range=15,    # randomly rotate images in the range (degrees, 0 to 180)
                            zoom_range = 0.1,       # Randomly zoom image
                            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                            height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
                            horizontal_flip=False,  # randomly flip images
                            vertical_flip=False)    # randomly flip images

valIDG = ImageDataGenerator()

In [ ]:
## ImageDataGenerator 설정에 따라 불필요 할 수 있다
# trainIDG.fit(train_x)
# valIDG.fit(val_x)

In [ ]:
flow_trainIDG = trainIDG.flow(train_x, train_y, batch_size=256)
flow_valIDG = valIDG.flow(val_x, val_y, batch_size=256)

## Modeling : CNN

- 조건
    1. Sequential API, Functiona API 중 택일.
    2. [이 구조를 미니 버전으로 활용해봐도 좋다.](https://img1.daumcdn.net/thumb/R800x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F99DFA5415B38AC752E)
    3. DropOut, BatchNormalization 등의 기능도 같이 활용해보자.
    4. Early Stopping을 사용할 것.

* Sequential API

In [ ]:
# 1. 세션 클리어
clear_session()

# 2. 모델 선언
model1 = Sequential()

# 3. 레이어 블록 조립
model1.add( Input(shape=(28,28,1)) )

model1.add( Conv2D(filters=64,          # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
                   kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
                   strides=(1,1),       # Conv2D 필터의 이동 보폭
                   padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
                   activation='relu',   # 빼먹지 않게 주의!
                   ) )
model1.add( Conv2D(filters=64,          # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
                   kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
                   strides=(1,1),       # Conv2D 필터의 이동 보폭
                   padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
                   activation='relu',   # 빼먹지 않게 주의!
                   ) )
model1.add( MaxPool2D(pool_size=(2,2),  # Maxpool2D 필터의 가로 세로 사이즈
                      strides=(2,2)     # Maxpool2D 필터의 이동 보폭 : 기본적으로 pool_size를 따른다.
                      ) )
model1.add( BatchNormalization() )
model1.add( Dropout(0.2) )

model1.add( Conv2D(filters=128,         # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
                   kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
                   strides=(1,1),       # Conv2D 필터의 이동 보폭
                   padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
                   activation='relu',   # 빼먹지 않게 주의!
                   ) )
model1.add( Conv2D(filters=128,         # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
                   kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
                   strides=(1,1),       # Conv2D 필터의 이동 보폭
                   padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
                   activation='relu',   # 빼먹지 않게 주의!
                   ) )
model1.add( MaxPool2D(pool_size=(2,2),  # Maxpool2D 필터의 가로 세로 사이즈
                      strides=(2,2)     # Maxpool2D 필터의 이동 보폭 : 기본적으로 pool_size를 따른다.
                      ) )
model1.add( BatchNormalization() )
model1.add( Dropout(0.2) )

model1.add( Flatten() )
model1.add( Dense(256, activation='relu') )
model1.add( Dense(10, activation='softmax') )

# 4. 컴파일
model1.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

model1.summary()

* Functional API

In [ ]:
# 1. 세션 클리어
clear_session()

# 2. 레이어 엮기
il = Input(shape=(28,28,1))

hl = Conv2D(filters=64,          # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
            kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
            strides=(1,1),       # Conv2D 필터의 이동 보폭
            padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
            activation='relu',   # 빼먹지 않게 주의!
            )(il)
hl = Conv2D(filters=64,          # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
            kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
            strides=(1,1),       # Conv2D 필터의 이동 보폭
            padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
            activation='relu',   # 빼먹지 않게 주의!
            )(hl)
hl = MaxPool2D(pool_size=(2,2),  # Maxpool2D 필터의 가로 세로 사이즈
               strides=(2,2)     # Maxpool2D 필터의 이동 보폭 : 기본적으로 pool_size를 따른다.
               )(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)

hl = Conv2D(filters=128,         # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
            kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
            strides=(1,1),       # Conv2D 필터의 이동 보폭
            padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
            activation='relu',   # 빼먹지 않게 주의!
            )(hl)
hl = Conv2D(filters=128,         # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
            kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
            strides=(1,1),       # Conv2D 필터의 이동 보폭
            padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
            activation='relu',   # 빼먹지 않게 주의!
            )(hl)
hl = MaxPool2D(pool_size=(2,2),  # Maxpool2D 필터의 가로 세로 사이즈
               strides=(2,2)     # Maxpool2D 필터의 이동 보폭 : 기본적으로 pool_size를 따른다.
               )(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)

hl = Flatten()(hl)
hl = Dense(256, activation='relu')(hl)
ol = Dense(10, activation='softmax')(hl)

# 3. 모델의 시작과 끝 지정
model2 = Model(il, ol)

# 4. 컴파일
model2.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

model2.summary()

* Early Stopping

In [ ]:
es = EarlyStopping(monitor='val_loss',         # 얼리 스토핑을 적용할 관측 대상
                   min_delta=0,                # Threshold. 설정한 값 이상으로 변화해야 성능이 개선되었다고 간주.
                   patience=3,                 # 성능 개선이 발생하지 않았을 때, 몇 epoch를 더 지켜볼 것인가
                   verbose=1,
                   restore_best_weights=True)  # 성능이 가장 좋은 epoch의 가중치를 적용함.

* **Model Checkpoint**

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
mcp = ModelCheckpoint(filepath='/content/model1.h5',   # 모델 저장 경로
                      monitor='val_loss',              # 모델 저장의 관심 대상
                      verbose=1,                       # 어느 시점에서 저장되는지 알려줌
                      save_best_only=True,             # 최고 성능 모델만 저장
                      save_weights_only=False)         # True : 가중치만 저장 | False : 모델 구조 포함하여 저장

* .fit( )

In [ ]:
history = model1.fit(flow_trainIDG, epochs=10000, verbose=1,
                     validation_data=flow_valIDG,
                     callbacks=[es, mcp]
                     )

In [ ]:
performance_test = model1.evaluate(test_x, test_y, batch_size=256)

print(f'Test Loss: {performance_test[0]:.6f}')
print(f'Test Accuracy: {performance_test[1]*100:.3f}%')

In [ ]:
if not isinstance(history, dict):
    history = history.history

plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('Accuracy : Training vs Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)
plt.show()

In [ ]:
if not isinstance(history, dict):
    history = history.history

plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Loss : Training vs Validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)
plt.show()

## **Model Save & Load**

- .save( )
- .load_model( )

* 모델을 새롭게 저장하여 구조와 가중치 일부를 살펴보자

In [ ]:
model1.save('my_first_save.h5')

In [ ]:
clear_session()
model = keras.models.load_model('my_first_save.h5')
model.summary()

In [ ]:
model.get_weights()[0][0][0]

* 학습 과정에서 저장된 모델을 불러와 구조와 가중치 일부를 살펴보자

In [ ]:
clear_session()
model = keras.models.load_model('/content/model1.h5')
model.summary()

In [ ]:
model.get_weights()[0][0][0]

* .predict( )

In [ ]:
pred_train = model.predict(train_x)
pred_test = model.predict(test_x)

single_pred_train = pred_train.argmax(axis=1)
single_pred_test = pred_test.argmax(axis=1)

logi_train_accuracy = accuracy_score(train_y.argmax(axis=1), single_pred_train)
logi_test_accuracy = accuracy_score(test_y.argmax(axis=1), single_pred_test)

print('CNN')
print(f'트레이닝 정확도 : {logi_train_accuracy*100:.2f}%')
print(f'테스트 정확도 : {logi_test_accuracy*100:.2f}%')

## Visualization

* 실제 데이터 확인

In [ ]:
'''
성능 확인을 위해
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = np.random.randint(0,10000)

print(f'id = {id}')
print(f'다음 그림은 숫자 {test_y.argmax(axis=1)[id]} 입니다.')
print(f'모델의 예측 : {single_pred_test[id]}')
print(f'모델의 카테고리별 확률 : {np.floor(pred_test[id]*100)}')

if test_y.argmax(axis=1)[id] == single_pred_test[id] :
    print('정답입니다')
else :
    print('틀렸어요')

plt.imshow(test_x[id].reshape([28,-1]), cmap='Greys')
plt.show()

* 틀린 이미지만 확인해보기

In [ ]:
true_false = (test_y.argmax(axis=1) == single_pred_test)
f_id = np.where(true_false == False)[0]
f_n = len(f_id)

id = f_id[np.random.randint(0,f_n)]

print(f'id = {id}')
print(f'다음 그림은 숫자 {test_y.argmax(axis=1)[id]} 입니다.')
print(f'모델의 예측 : {single_pred_test[id]}')
print(f'모델의 카테고리별 확률 : {np.floor(pred_test[id]*100)}')

if test_y.argmax(axis=1)[id] == single_pred_test[id] :
    print('정답입니다')
else :
    print('틀렸어요')

plt.imshow(test_x[id].reshape([28,-1]), cmap='Greys')
plt.show()

# **내가 만든 손글씨 이미지는 어떻게 판단할까?**
---
## **구글 드라이브에 손글씨 이미지를 업로드!**
###**순서**
1. 그림판으로 숫자를 그려서 저장한다.
2. 구글 드라이브 첫 화면에 my_data 라는 폴더를 만든다.
3. my_data 폴더 안에 my_mnist 폴더를 만든다.
4. my_mnist 폴더 안에 1번 과정에서 만든 이미지를 업로드한다.
5. 30초 정도 기다립시다.
6. 아래의 코드들을 실행해본다.

## Connect Colaboratory with my Google Drive
- Colaboratory와 본인의 구글 드라이브를 연결하는 과정
- 아래 코드를 실행하여 폴더가 올바르게 생성 되었는지 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
!cd /content/drive/MyDrive/my_data; ls

## Load Image
- 업로드 한 이미지 하나를 불러와 확인해본다

In [ ]:
import glob
from tensorflow.keras.preprocessing import image

In [ ]:
files = glob.glob('/content/drive/MyDrive/my_data/my_mnist/*')
files

In [ ]:
img = image.load_img(files[0], color_mode='grayscale', target_size=(28,28) )
img = image.img_to_array(img)

plt.imshow(img.reshape(img.shape[0], img.shape[1]), cmap='gray')
plt.show()

In [ ]:
img = 255-img

In [ ]:
plt.imshow(img.reshape(img.shape[0],img.shape[1]), cmap='Greys'  )
plt.show()

In [ ]:
model.predict(img.reshape((-1,28,28,1)))[0].argmax()

## Load Images
- 업로드 한 이미지 전체를 확인해본다

In [ ]:
images = []

for path in files :
    img = image.load_img(path, color_mode='grayscale', target_size=(28,28) )
    img = image.img_to_array(img)
    img = 255-img
    images.append(img)

images = np.array(images)

In [ ]:
images.shape

In [ ]:
pred = model.predict(images)

for i in range(images.shape[0]) :
    print('====================================')
    print(f'모델의 예측 : {pred[i].argmax()}')
    print(f'모델의 카테고리별 확률 : {np.floor(pred[i]*100)}')

    plt.imshow(images[i].reshape(28,28) , cmap='Greys')
    plt.show()

---

# Extra: flow_from_directory
## 이미 내가 이미지 데이터를 클래스별로 정리한 상태라면?
## 그럼에도 이미지 데이터에 대한 Augmentation이 필요하다면?
---
## **순서**
1. 구글 드라이브에 my_data/my_mnist2 폴더를 생성합니다.
2. my_mnist2 폴더 안에 손글씨가 폴더별로 구분되어야 한다.
    - ex) 0에 대한 이미지면 my_data/my_mnist2/0/0_1.jpg
    - ex) 1에 대한 이미지면 my_data/my_mnist2/1/1_1.jpg

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_idg = ImageDataGenerator(rotation_range=25,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               vertical_flip=True,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               )

val_idg = ImageDataGenerator()

In [ ]:
!cd /content/drive/MyDrive/my_data; ls

In [ ]:
!mkdir /content/drive/MyDrive/my_data/temp/

In [ ]:
!cd /content/drive/MyDrive/my_data; ls

In [ ]:
flow_dir_trainIDG = train_idg.flow_from_directory('/content/drive/MyDrive/my_data/my_mnist2',
                                                  save_to_dir='/content/drive/MyDrive/my_data/temp/',
                                                  save_prefix='train',
                                                  save_format='jpg',
                                                  target_size=(28,28),
                                                  color_mode='grayscale',
                                                  class_mode='categorical'
                                                  )

flow_dir_valIDG = val_idg.flow_from_directory('/content/drive/MyDrive/my_data/my_mnist2',
                                              save_to_dir='/content/drive/MyDrive/my_data/temp/',
                                              save_prefix='val',
                                              save_format='jpg',
                                              target_size=(28,28),
                                              color_mode='grayscale',
                                              class_mode='categorical'
                                              )

In [ ]:
clear_session()

model = keras.models.load_model('/content/model1.h5')

model.summary()

In [ ]:
model.fit(flow_dir_trainIDG, validation_data=flow_dir_valIDG,
          epochs=100, verbose=1, callbacks=[es])